In [ ]:
# ubuntu server with +8GB mem, +20GB disk (first sync is CPU intensive)
# open a putty session with ubuntu server
# download, install and run the cardano-wallet backend node using docker
# wget https://github.com/input-output-hk/cardano-wallet/raw/master/docker-compose.yml
# NETWORK=mainnet docker-compose up -d

In [1]:
import requests

In [2]:
requests.get('http://localhost:8090/v2/network/information')

<Response [200]>

In [5]:
# wait for the node status:ready (finish syncing the blockchain...)
requests.get('http://localhost:8090/v2/network/information').text

'{"node_era":"byron","node_tip":{"height":{"quantity":530244,"unit":"block"},"time":"2018-01-24T15:54:31Z","epoch_number":24,"absolute_slot_number":530309,"slot_number":11909},"sync_progress":{"status":"syncing","progress":{"quantity":8.27,"unit":"percent"}}}'

In [7]:
# check on the API if a wallet already exist on this server, [] means empty
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/getWallet
requests.get('http://localhost:8090/v2/wallets').text

'[]'

In [8]:
# create a new wallet with the API
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/postWallet
# the mnemonic words is the private seed secret 
requests.post('http://localhost:8090/v2/wallets', json={"name":"Kevin's wallet", "mnemonic_sentence":["pride","assault","husband","ball","convince","canvas","mind","crane","nurse","feel","tongue","pride","scrap","inflict","split","antenna","slight","foster","piano","carpet","ranch","unveil","believe","black"], "passphrase":"ABCD1234abcd", "address_pool_gap":20}).text

'{"passphrase":{"last_updated_at":"2021-10-17T17:11:47.341645525Z"},"address_pool_gap":20,"state":{"status":"syncing","progress":{"quantity":0,"unit":"percent"}},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":0,"unit":"lovelace"},"available":{"quantity":0,"unit":"lovelace"}},"name":"Kevin\'s wallet","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":0,"unit":"block"},"time":"2017-09-23T21:44:51Z","epoch_number":0,"absolute_slot_number":0,"slot_number":0},"assets":{"total":[],"available":[]}}'

In [11]:
# check wallets
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/getWallet
requests.get('http://localhost:8090/v2/wallets').text

'[{"passphrase":{"last_updated_at":"2021-10-17T17:11:47.341645525Z"},"address_pool_gap":20,"state":{"status":"syncing","progress":{"quantity":24.98,"unit":"percent"}},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":0,"unit":"lovelace"},"available":{"quantity":0,"unit":"lovelace"}},"name":"Kevin\'s wallet","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":1601885,"unit":"block"},"time":"2018-09-29T19:04:31Z","epoch_number":74,"absolute_slot_number":1602239,"slot_number":3839},"assets":{"total":[],"available":[]}}]'

In [ ]:
# grab the wallet "id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c"

In [2]:
import json

In [13]:
# show a list of known addresses, ordered from newest to oldest generated from our main wallet id
# https://input-output-hk.github.io/cardano-wallet/api/edge/#tag/Addresses
requests.get('http://localhost:8090/v2/wallets/34d88843c44eb8b13a0f03a3ef387fa3b5e8733c/addresses').text

'[{"state":"unused","id":"addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv","derivation_path":["1852H","1815H","0H","0","0"]},{"state":"unused","id":"addr1q9dsnjj2ryz55ykq9kwp48hvx4cfqtfszna3lqnuzj29feaucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs20wk26","derivation_path":["1852H","1815H","0H","0","1"]},{"state":"unused","id":"addr1q8sazmfqsad2hm4j3052dj4rhlh8jn7ft2yy8z8m85al8lducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsy40dr5","derivation_path":["1852H","1815H","0H","0","2"]},{"state":"unused","id":"addr1q95k6rp7jyp0zhdm240v2rsnmgewnxhy57h8ky3aaduw0s4ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhss2rly5","derivation_path":["1852H","1815H","0H","0","3"]},{"state":"unused","id":"addr1qxfht2ar6fydjf0n27ccc796zmqmqk244mmpvp06ww95uf4ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs8zp0as","derivation_path":["1852H","1815H","0H","0","4"]},{"state":"unused","id":"addr1qym3t8e9462k7mvhargrxs0hn2dn3kpwdrt5kz3tr6d7cc4ucnhmu76m8kpkr24jwzc95emzqq

In [14]:
# transfer any amount of ADA from mainnet to this address picked from the list above:
# addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv

# using Daedalus or Yoroi or any other wallet I just send 3 ADA... (-0.8 fee) = 2.2 ADA
# wait for all systems to be synced... ~24 hours

vanaf hier verder gaan morgen

In [9]:
# check available quantity 2200000 lovelace
# check wallets
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/getWallet
requests.get('http://localhost:8090/v2/wallets').text

'[{"passphrase":{"last_updated_at":"2021-10-17T17:11:47.341645525Z"},"address_pool_gap":20,"state":{"status":"ready"},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":2200000,"unit":"lovelace"},"available":{"quantity":2200000,"unit":"lovelace"}},"name":"Kevin\'s wallet","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":6386269,"unit":"block"},"time":"2021-10-18T10:20:45Z","epoch_number":297,"absolute_slot_number":42986154,"slot_number":45354},"assets":{"total":[],"available":[]}}]'

In [13]:
# check transaction fee (estimation value)
# thi only workes if the blockchain is in sync and the funds are available
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/postTransactionFee
# 1 ADA = 1.000.000 lovelace
transaction_data = {"passphrase":"ABCD1234abcd", "payments":[{"address":"Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7", "amount":{"quantity":2200000, "unit":"lovelace"}}]}
requests.post('http://localhost:8090/v2/wallets/34d88843c44eb8b13a0f03a3ef387fa3b5e8733c/payment-fees', json=transaction_data).text

'{"estimated_min":{"quantity":168405,"unit":"lovelace"},"deposit":{"quantity":0,"unit":"lovelace"},"minimum_coins":[{"quantity":999978,"unit":"lovelace"}],"estimated_max":{"quantity":168405,"unit":"lovelace"}}'

In [14]:
# execute transaction to another wallet (return the funds exercise)
# fill in the fee that was calculated, then calculate the remaining amount to send available
fee = 168405
quantity = 2200000 - fee
quantity

2031595

In [16]:
# send transaction from the wallet
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/postTransaction
transaction_data = {"passphrase":"ABCD1234abcd", "payments":[{"address":"Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7", "amount":{"quantity":quantity, "unit":"lovelace"}}]}
requests.post('http://localhost:8090/v2/wallets/34d88843c44eb8b13a0f03a3ef387fa3b5e8733c/transactions', json=transaction_data).text

'{"status":"pending","withdrawals":[],"amount":{"quantity":2200000,"unit":"lovelace"},"inputs":[{"amount":{"quantity":2200000,"unit":"lovelace"},"address":"addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv","id":"94c08d753539cd17865b9d429718cf80fb9dc49251a628931a1b31750651a580","assets":[],"index":2}],"direction":"outgoing","fee":{"quantity":168405,"unit":"lovelace"},"outputs":[{"amount":{"quantity":2031595,"unit":"lovelace"},"address":"Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7","assets":[]}],"script_validity":"valid","expires_at":{"time":"2021-10-18T12:32:39Z","epoch_number":297,"absolute_slot_number":42994068,"slot_number":53268},"pending_since":{"height":{"quantity":6386312,"unit":"block"},"time":"2021-10-18T10:32:27Z","epoch_number":297,"absolute_slot_number":42986856,"slot_number":46056},"metadata":null,"id":"a61810a8f9ad2253b753b30dcdd03b4c6759156b7eaa5bf0db2d194ab9a7b915","deposit":{"quantity":0,"unit":"love

In [17]:
# check to see the available balance to be 0 lovelace if success in sending away all ADA
# check wallets
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/getWallet
requests.get('http://localhost:8090/v2/wallets').text

'[{"passphrase":{"last_updated_at":"2021-10-17T17:11:47.341645525Z"},"address_pool_gap":20,"state":{"status":"ready"},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":0,"unit":"lovelace"},"available":{"quantity":0,"unit":"lovelace"}},"name":"Kevin\'s wallet","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":6386316,"unit":"block"},"time":"2021-10-18T10:33:42Z","epoch_number":297,"absolute_slot_number":42986931,"slot_number":46131},"assets":{"total":[],"available":[]}}]'

work this example all over using this library

In [ ]:
# you could replace all of this with this tool
# https://cardano-python.readthedocs.io/en/latest/wallet.html#creating-wallets